<a href="https://colab.research.google.com/github/Rajat-Kumar-Pandey/MACHINE-LEARNING/blob/main/compleete_drowsy_modal_with_best_accurrecy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, Callback
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from PIL import Image
import logging
import os
import json

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

logger.info("Script execution started.")

# Enable mixed precision for faster training on GPUs
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

# Check if GPU (CUDA) is available and log the device being used
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    logger.info(f"GPU available: {physical_devices}")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    logger.warning("No GPU found, using CPU instead.")

# Set dataset path
dataset_path = '/content/drive/MyDrive/Drowssinness data/Resized_Driver_Drowsiness_Dataset'

# Check and remove corrupted images
def remove_corrupted_images(directory):
    logger.info("Checking for corrupted images...")
    for root, _, files in os.walk(directory):
        for file in files:
            try:
                with Image.open(os.path.join(root, file)) as img:
                    img.verify()
            except Exception:
                os.remove(os.path.join(root, file))
                logger.info(f"Removed corrupted file: {os.path.join(root, file)}")

remove_corrupted_images(dataset_path)

# Constants
SEED = 123
IMAGE_SIZE = 256
CHANNELS = 3
BATCH_SIZE = 32
EPOCHS = 30
N_CLASSES = 2

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])
logger.info("Data augmentation pipeline initialized.")

# Load datasets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset="training"
).map(lambda x, y: (data_augmentation(x), y)).prefetch(buffer_size=tf.data.AUTOTUNE)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset="validation"
).prefetch(buffer_size=tf.data.AUTOTUNE)

# Build CNN Model
def build_cnn_model():
    model = models.Sequential([
        layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(N_CLASSES, activation='softmax')
    ])
    return model

cnn_model = build_cnn_model()
cnn_model.summary()

# Save the model immediately after building it
cnn_model.save('cnn_drowsiness_baseline_model.h5')
logger.info("Baseline model saved as 'cnn_drowsiness_baseline_model.h5'.")

# Compile the model
cnn_model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Custom callback to save the model when validation accuracy exceeds 98%
class SaveHighAccuracyModel(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') > 0.98:
            logger.info(f"Validation accuracy exceeded 98% at epoch {epoch + 1}. Saving model...")
            self.model.save(f'cnn_drowsiness_high_accuracy_model_epoch_{epoch + 1}.h5')

callbacks = [
    ModelCheckpoint('cnn_drowsiness_best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    TensorBoard(log_dir='./logs', histogram_freq=1),
    SaveHighAccuracyModel()
]

# Train the model
logger.info("Training the CNN model...")
history = cnn_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

# Save the final model and training history
cnn_model.save('cnn_drowsiness_final_model.h5')
logger.info("Final model saved as 'cnn_drowsiness_final_model.h5'.")
with open('training_history.json', 'w') as f:
    json.dump(history.history, f)
logger.info("Training history saved.")
